In [6]:
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.conf import SparkConf
from pyspark.sql.functions import unix_timestamp


spark = SparkSession.builder.config('spark.executor.cores','2').master("local").appName("RankTimeSeries").getOrCreate()

# 11809233 


In [7]:
# Rank by UnixTime('Issue Date')
no_chunks = 1
year = 2010

df = spark.read.format("csv").option("header", "true").load('data/2010_Census_Populations_by_Zip_Code.csv')

Dates = df.select('Total Population', 'Total Households').fillna(0)
oldColumns = Dates.schema.names
newColumns = ["tot_households", "tot_population"]
Dates = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), Dates)
#Dates.printSchema()
#Dates.show()
Dates.createOrReplaceTempView("initial")
Ranked = spark.sql("""
    SELECT i.tot_population, ROW_NUMBER() OVER (ORDER BY i.tot_households ASC) AS position, FLOOR(RAND()*{number_of_chunks}) as chunk
    FROM initial as i
    """.format(number_of_chunks = no_chunks))
Ranked.createOrReplaceTempView("initial")


Chunk0 = spark.sql("""
    SELECT i.tot_population, i.position
    FROM initial as i
    WHERE i.chunk = 0
""")

#Ranked = Dates.withColumn("position", rank().over(window))
#Ranked.coalesce(1)

#Ranked.write.format('com.databricks.spark.csv').save('data/ranked',mode='overwrite')



In [4]:

X = Chunk0.rdd.collect() 

In [8]:
Chunk0.coalesce(1).write.csv('data/preprocessed_census_ASC_{year}'.format(year=year),header = 'true')

In [5]:
X

[Row(tot_population='3427', position=1),
 Row(tot_population='23278', position=2),
 Row(tot_population='3615', position=3),
 Row(tot_population='26056', position=4),
 Row(tot_population='31087', position=5),
 Row(tot_population='23543', position=6),
 Row(tot_population='25144', position=7),
 Row(tot_population='18648', position=8),
 Row(tot_population='3706', position=9),
 Row(tot_population='3385', position=10),
 Row(tot_population='21952', position=11),
 Row(tot_population='2729', position=12),
 Row(tot_population='3371', position=13),
 Row(tot_population='23650', position=14),
 Row(tot_population='17504', position=15),
 Row(tot_population='18419', position=16),
 Row(tot_population='20864', position=17),
 Row(tot_population='22940', position=18),
 Row(tot_population='22855', position=19),
 Row(tot_population='1982', position=20),
 Row(tot_population='20672', position=21),
 Row(tot_population='17708', position=22),
 Row(tot_population='20964', position=23),
 Row(tot_population='4109',